In [4]:
# !pip install psycopg2
from sqlalchemy import create_engine
import os
from os import listdir
from os.path import isfile, join
from datetime import datetime
import pandas as pd
import numpy as np

from settings import *

In [6]:
data_dir = os.path.join(os.path.abspath(''), '../../datasets/raw/')

pop_data_dir = os.path.join(os.path.abspath(''), '../../datasets/datasus/')

In [7]:
# obtém lista de arquivos
files = [f for f in listdir(data_dir) if isfile(join(data_dir, f)) and f.startswith('open-datasus')]
files.sort()


# obtém sufixo (data) dos arquivos mais recentes
suffix = files[-1].split('-')[2]

# seleciona arquivos mais recentes
last_files = [f for f in files if f.endswith(suffix)]

last_files

['open-datasus_AC-2021_05_16.csv',
 'open-datasus_AL-2021_05_16.csv',
 'open-datasus_AM-2021_05_16.csv',
 'open-datasus_AP-2021_05_16.csv',
 'open-datasus_BA-2021_05_16.csv',
 'open-datasus_CE-2021_05_16.csv',
 'open-datasus_DF-2021_05_16.csv',
 'open-datasus_ES-2021_05_16.csv',
 'open-datasus_GO-2021_05_16.csv',
 'open-datasus_MA-2021_05_16.csv',
 'open-datasus_MG-2021_05_16.csv',
 'open-datasus_MS-2021_05_16.csv',
 'open-datasus_MT-2021_05_16.csv',
 'open-datasus_PA-2021_05_16.csv',
 'open-datasus_PB-2021_05_16.csv',
 'open-datasus_PE-2021_05_16.csv',
 'open-datasus_PI-2021_05_16.csv',
 'open-datasus_PR-2021_05_16.csv',
 'open-datasus_RJ-2021_05_16.csv',
 'open-datasus_RN-2021_05_16.csv',
 'open-datasus_RO-2021_05_16.csv',
 'open-datasus_RR-2021_05_16.csv',
 'open-datasus_RS-2021_05_16.csv',
 'open-datasus_SC-2021_05_16.csv',
 'open-datasus_SE-2021_05_16.csv',
 'open-datasus_SP-2021_05_16.csv',
 'open-datasus_TO-2021_05_16.csv']

In [4]:
file = last_files[1]

file

'open-datasus_AL-2021_05_16.csv'

In [5]:
FIELD_LIST = [
    'paciente_id',
    'paciente_idade',
    'paciente_enumSexoBiologico',
    'paciente_racaCor_valor',
    'paciente_endereco_cep',
    'paciente_endereco_coIbgeMunicipio',
    'estabelecimento_municipio_codigo',
    'estabelecimento_municipio_nome',
    'estabelecimento_uf',
    'vacina_grupoAtendimento_nome',
    'vacina_categoria_codigo',
    'vacina_categoria_nome',
    'vacina_lote',
    'vacina_dataAplicacao',
    'vacina_descricao_dose',
    'vacina_nome',
    'sistema_origem',
    'data_importacao_rnds'
]

FIELD_TYPE_LIST = {
    'paciente_id': 'str',
    'paciente_idade': np.uint8,
    'paciente_enumSexoBiologico': 'str',
    'paciente_racaCor_valor': 'str',
    'paciente_endereco_coIbgeMunicipio': 'int',
    'estabelecimento_municipio_codigo': 'int',
    'estabelecimento_municipio_nome': 'str',
    'estabelecimento_uf': 'str',
    'vacina_grupoAtendimento_nome': 'str',
    'vacina_categoria_nome': 'str',
    'vacina_lote': 'str',
    'vacina_dataAplicacao': 'str',
    'vacina_descricao_dose': 'str',
    'vacina_nome': 'str',
    'sistema_origem': 'str',
    'data_importacao_rnds': 'object'
}

In [6]:
FIELD_TYPE_LIST = {col['nome']: col['tipo_pandas'] for col in COLUNAS_VACINACAO if col['tipo'] not in ['date']}
FIELD_TYPE_LIST

{'paciente_id': 'str',
 'paciente_idade': 'int64',
 'paciente_enumsexobiologico': 'str',
 'paciente_racacor_valor': 'str',
 'paciente_endereco_coibgemunicipio': Int64Dtype(),
 'paciente_endereco_uf': 'str',
 'paciente_endereco_cep': Int64Dtype(),
 'paciente_nacionalidade_enumnacionalidade': 'str',
 'estabelecimento_valor': 'int64',
 'estabelecimento_razaosocial': 'str',
 'estalecimento_nofantasia': 'str',
 'estabelecimento_municipio_codigo': Int64Dtype(),
 'estabelecimento_uf': 'str',
 'vacina_grupoatendimento_nome': 'str',
 'vacina_categoria_codigo': Int64Dtype(),
 'vacina_categoria_nome': 'str',
 'vacina_descricao_dose': 'str',
 'vacina_nome': 'str',
 'sistema_origem': 'str'}

In [7]:
df = pd.read_csv(data_dir + file, dtype=FIELD_TYPE_LIST,sep=';', parse_dates=COLUNAS_DATA) #dtype=FIELD_TYPE_LIST, 

df

,document_id,paciente_id,paciente_idade,paciente_datanascimento,paciente_enumsexobiologico,paciente_racacor_codigo,paciente_racacor_valor,paciente_endereco_coibgemunicipio,paciente_endereco_copais,paciente_endereco_nmmunicipio,...,vacina_lote,vacina_fabricante_nome,vacina_fabricante_referencia,vacina_dataaplicacao,vacina_descricao_dose,vacina_codigo,vacina_nome,sistema_origem,data_importacao_rnds,id_sistema_origem
0,9df581a6-38cb-4d22-9b64-de8c2403c38e-i0b0,90f7e07244966c6e990f9302d605edf604de77c65ad1ee...,47,1974-01-17,F,1,BRANCA,280570,10.0,PROPRIA,...,214VCD052W,FUNDACAO OSWALDO CRUZ,Organization/33781055000135,2021-05-12,1ª Dose,85,Vacina Covid-19 - Covishield,Novo PNI,2021-05-15 01:51:19+00:00,NaN
1,d9ef97bc-20b4-4850-b1bb-ed10b5dc68fb-i0b0,c5a2ea88a3d3d71cdd92b679da23c0866e5d2bb80929b2...,66,1955-03-08,M,99,SEM INFORMACAO,270430,10.0,MACEIO,...,210141,FUNDACAO BUTANTAN,Organization/61189445000156,2021-04-23,2ª Dose,86,Covid-19-Coronavac-Sinovac/Butantan,Novo PNI,2021-04-23 14:35:29+00:00,NaN
2,7d9f033c-e4dc-469a-925e-314ae2fcd41f-i0b0,d272e3df2ec45d059c6881dcca96fc1aea6ac6c897f552...,89,1932-05-06,M,1,BRANCA,270255,10.0,ESTRELA DE ALAGOAS,...,213VCD009VA,FUNDACAO OSWALDO CRUZ,Organization/33781055000135,2021-05-10,2ª Dose,85,Vacina Covid-19 - Covishield,Novo PNI,2021-05-15 01:29:07+00:00,NaN
3,b07684b8-11d6-4d31-88bb-6c5fe5eda31c-i0b0,e9a1ac482b77778ebf5c51f5f354e5495329578d3197b2...,71,1949-04-23,F,1,BRANCA,270630,10.0,PALMEIRA DOS INDIOS,...,210110,FUNDACAO BUTANTAN,Organization/61189445000156,2021-04-20,2ª Dose,86,Covid-19-Coronavac-Sinovac/Butantan,Novo PNI,2021-04-23 02:47:17+00:00,NaN
4,5dfd9321-6708-4ef1-a268-6e489890f242-i0b0,e1473ef9f30258c8fdb27a764af5c38f3b27493222ab02...,89,1931-12-23,F,1,BRANCA,270255,10.0,ESTRELA DE ALAGOAS,...,213VCD009VA,FUNDACAO OSWALDO CRUZ,Organization/33781055000135,2021-05-10,2ª Dose,85,Vacina Covid-19 - Covishield,Novo PNI,2021-05-15 01:24:57+00:00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
776084,971a97a8-0ec1-4cbf-b84d-fb7313e8bd55-i0b0,ba07825b8510a1482fd9f79a5d73d6376d8b659c37cc1f...,89,1931-06-18,F,99,SEM INFORMACAO,270430,10.0,MACEIO,...,213VCD007ZVA,FUNDACAO OSWALDO CRUZ,Organization/33781055000135,2021-04-22,2ª Dose,85,Vacina Covid-19 - Covishield,Novo PNI,2021-04-23 09:08:06+00:00,NaN
776085,896e53af-c1cf-4400-b60f-a31a598f796b-i0b0,6a0aad55e9b185baf5ef38d057fdf0fb5350923c1a469f...,72,1948-11-06,F,3,PARDA,270240,10.0,DELMIRO GOUVEIA,...,210041,FUNDACAO BUTANTAN,Organization/61189445000156,2021-04-15,2ª Dose,86,Covid-19-Coronavac-Sinovac/Butantan,Novo PNI,2021-04-16 18:08:03+00:00,NaN
776086,5c092f16-0b71-4379-b8bd-60259ce2cf7c-i0b0,e2aaab815a31ccca46f0fd0ce889fac5bf175612be5b14...,61,1959-11-05,M,1,BRANCA,270670,10.0,PENEDO,...,210150,FUNDACAO BUTANTAN,Organization/61189445000156,2021-05-05,2ª Dose,86,Covid-19-Coronavac-Sinovac/Butantan,Novo PNI,2021-05-05 14:50:11+00:00,NaN
776087,89930e2d-a9fe-437d-aa7f-568056624126-i0b0,f93349cd37aa94a7cef8872d023202077bac88581f1321...,61,1959-05-20,F,99,SEM INFORMACAO,270030,10.0,ARAPIRACA,...,210141,FUNDACAO BUTANTAN,Organization/61189445000156,2021-04-24,2ª Dose,86,Covid-19-Coronavac-Sinovac/Butantan,Novo PNI,2021-04-24 14:03:22+00:00,NaN


In [8]:
df.describe(include='all')

/home/luizcelso/bin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  """Entry point for launching an IPython kernel.
/home/luizcelso/bin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  """Entry point for launching an IPython kernel.
/home/luizcelso/bin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Spec

,document_id,paciente_id,paciente_idade,paciente_datanascimento,paciente_enumsexobiologico,paciente_racacor_codigo,paciente_racacor_valor,paciente_endereco_coibgemunicipio,paciente_endereco_copais,paciente_endereco_nmmunicipio,...,vacina_lote,vacina_fabricante_nome,vacina_fabricante_referencia,vacina_dataaplicacao,vacina_descricao_dose,vacina_codigo,vacina_nome,sistema_origem,data_importacao_rnds,id_sistema_origem
count,776089,776089,776089.000000,776089,776089,776089.000000,776089,772844.000000,772885.000000,772848,...,776089,776089,774394,776089,776089,776089.000000,776089,776089,776089,0.0
unique,775677,536728,NaN,29699,2,NaN,6,NaN,NaN,1994,...,58,5,3,119,2,NaN,4,2,653125,NaN
top,3c5fee86-3692-4cf6-be0f-4e99e0e04ff3-i0b0,625b5b99f2b46e4e03a23e3600d66a4028f639ab0b5807...,NaN,1950-01-20 00:00:00,F,NaN,PARDA,NaN,NaN,MACEIO,...,210110,FUNDACAO BUTANTAN,Organization/61189445000156,2021-04-07 00:00:00,1ª Dose,NaN,Covid-19-Coronavac-Sinovac/Butantan,Novo PNI,2021-03-02 20:31:14+00:00,NaN
freq,2,5,NaN,215,454233,NaN,270953,NaN,NaN,248140,...,97598,505440,505440,20434,534908,NaN,506600,774394,322,NaN
first,NaN,NaN,NaN,1899-12-30 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2021-01-17 00:00:00,NaN,NaN,NaN,NaN,2021-01-23 21:57:51+00:00,NaN
last,NaN,NaN,NaN,2020-10-15 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2021-05-15 00:00:00,NaN,NaN,NaN,NaN,2021-05-15 15:07:15+00:00,NaN
mean,NaN,NaN,61.262293,NaN,NaN,20.589059,NaN,272707.752180,10.000849,NaN,...,NaN,NaN,NaN,NaN,NaN,85.871207,NaN,NaN,NaN,NaN
std,NaN,NaN,16.308692,NaN,NaN,37.345199,NaN,20771.062438,0.388111,NaN,...,NaN,NaN,NaN,NaN,NaN,0.871375,NaN,NaN,NaN,NaN
min,NaN,NaN,0.000000,NaN,NaN,1.000000,NaN,110001.000000,10.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,85.000000,NaN,NaN,NaN,NaN
25%,NaN,NaN,52.000000,NaN,NaN,3.000000,NaN,270310.000000,10.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,85.000000,NaN,NaN,NaN,NaN


In [9]:
engine = create_engine(connection_uri)

In [10]:


df.to_sql(
    name='staging',
    con=engine,
    if_exists='replace',
    index=False)

In [11]:
data_dir

'/home/luizcelso/Insync/GDriveUTFPR/PycharmProjects/painel-vacina/src/extract/../../datasets/raw/'

In [14]:
df_pop = pd.read_csv(pop_data_dir + 'faixas_niveis_2020.csv', sep=';')

df_pop

,nivel,CodEst,CodMun,Sexo,De 0 a 4 anos,De 5 a 9 anos,De 10 a 14 anos,De 15 a 19 anos,De 20 a 24 anos,De 25 a 29 anos,...,De 35 a 39 anos,De 40 a 44 anos,De 45 a 49 anos,De 50 a 54 anos,De 55 a 59 anos,De 60 a 64 anos,De 65 a 69 anos,De 70 a 74 anos,De 75 a 79 anos,De 80 anos ou mais
0,N,NaN,NaN,F,7193517,7158663,7241977,7748675,8520327,8501472,...,8679141,8028154,7076092,6576798,5951176,5033166,4012245,3009884,2072284,2752002
1,N,NaN,NaN,M,7536783,7491621,7563503,8042215,8712946,8484387,...,8347424,7574841,6576416,6041004,5306094,4350558,3336996,2398773,1542100,1689044
2,E,11.0,NaN,F,68339,66378,67020,73504,77092,78121,...,74691,67166,58596,50381,41457,31993,23135,14901,9485,9404
3,E,11.0,NaN,M,71586,69560,70266,76484,79509,78871,...,74905,66545,56831,51997,43973,33258,23038,15009,9093,8822
4,E,12.0,NaN,F,40507,41477,43925,45538,43258,38432,...,33865,29686,23686,19599,15028,11958,9015,6382,4639,5080
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11191,M,52.0,522200.0,M,494,492,430,496,538,563,...,544,553,502,483,390,318,196,172,98,112
11192,M,52.0,522205.0,M,342,344,323,348,366,375,...,406,379,340,296,253,161,133,81,58,60
11193,M,52.0,522220.0,M,290,272,242,290,328,325,...,499,309,198,153,142,105,50,43,27,23
11194,M,52.0,522230.0,M,224,225,178,183,217,238,...,195,235,239,216,196,168,112,79,52,39


In [15]:
df_pop.to_sql(
    name='populacao',
    con=engine,
    if_exists='fail',
    index=False)